<a href="https://colab.research.google.com/github/MEROO1010/Emotion-Based-Music-Recommender/blob/main/Emotion_Based_Music_Recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Step 1: Install Required Libraries**

In [1]:
!pip install deepface spotipy streamlit opencv-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.3/128.3 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 108.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 57.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 91.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 60.9 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.7.0-py3-none-any.whl size=114249 sha256=71feb478a792355a4b7cd6465437f877c15a6aac8ebe6b21b422061ca304ebc0
  Stored in directory: /r

# **Step 2: Import Libraries**

In [2]:
import cv2
from deepface import DeepFace
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import streamlit as st
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, clear_output

25-08-06 00:26:41 - Directory /root/.deepface has been created
25-08-06 00:26:41 - Directory /root/.deepface/weights has been created


# **Step 3: Set Up Spotify API**

In [3]:
# Spotify API credentials (replace with yours)
SPOTIFY_CLIENT_ID = "your_client_id"
SPOTIFY_CLIENT_SECRET = "your_client_secret"
SPOTIFY_REDIRECT_URI = "http://localhost:8888/callback"

# Initialize Spotify client
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(
    client_id=SPOTIFY_CLIENT_ID,
    client_secret=SPOTIFY_CLIENT_SECRET,
    redirect_uri=SPOTIFY_REDIRECT_URI,
    scope="user-library-read playlist-modify-public"
))

# **Step 4: Define Emotion-Playlist Mapping**

In [4]:
# Emotion → Spotify Playlist IDs (replace with your playlists)
EMOTION_PLAYLISTS = {
    "happy": "37i9dQZF1DXdPec7aLTmlC",  # Happy Hits (Spotify)
    "sad": "37i9dQZF1DX7qK8ma5wgG1",    # Sad Songs
    "angry": "37i9dQZF1DX1s9knjP51Oa",   # Rock Workout
    "neutral": "37i9dQZF1DX4o1oenSJRJd", # Chill Vibes
    "surprise": "37i9dQZF1DX4mWCZw6qYIw", # Surprise Mix
    "fear": "37i9dQZF1DWYBO1MoTDhZI",    # Calm Meditation
    "disgust": "37i9dQZF1DX4WYpdgoIcn6"  # Lofi Beats
}

# **Step 5: Real-Time Emotion Detection & Music Recommendation**

In [5]:
# Open webcam
cap = cv2.VideoCapture(0)

# Create a figure for live updates
fig = plt.figure(figsize=(10, 6))
plt.axis('off')

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert BGR to RGB (DeepFace requires RGB)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    try:
        # Detect emotion using DeepFace
        emotion_result = DeepFace.analyze(rgb_frame, actions=['emotion'], enforce_detection=False)
        dominant_emotion = emotion_result[0]['dominant_emotion']

        # Display emotion on frame
        cv2.putText(frame, f"Emotion: {dominant_emotion}", (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        # Get Spotify playlist for detected emotion
        playlist_id = EMOTION_PLAYLISTS.get(dominant_emotion, EMOTION_PLAYLISTS["neutral"])
        playlist = sp.playlist(playlist_id)

        # Display playlist info
        cv2.putText(frame, f"Playlist: {playlist['name']}", (20, 100), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2)

    except Exception as e:
        print(f"Error: {e}")

    # Display the frame in Jupyter Notebook
    clear_output(wait=True)
    plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    display(fig)

    # Exit on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
plt.close()

# **Step 6: Deploy with Streamlit (Optional)**

In [6]:
import streamlit as st
import cv2
from deepface import DeepFace
import spotipy
from spotipy.oauth2 import SpotifyOAuth

# Spotify setup
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(
    client_id=SPOTIFY_CLIENT_ID,
    client_secret=SPOTIFY_CLIENT_SECRET,
    redirect_uri=SPOTIFY_REDIRECT_URI,
    scope="user-library-read"
))

st.title("🎵 Emotion-Based Music Recommender")

# Webcam capture
run = st.checkbox("Start Webcam")
FRAME_WINDOW = st.image([])
cap = cv2.VideoCapture(0)

while run:
    ret, frame = cap.read()
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    FRAME_WINDOW.image(frame)

    # Emotion detection & playlist recommendation
    try:
        emotion = DeepFace.analyze(frame, actions=['emotion'])['dominant_emotion']
        playlist_id = EMOTION_PLAYLISTS[emotion]
        playlist = sp.playlist(playlist_id)
        st.success(f"🎶 Recommended Playlist: {playlist['name']}")
    except:
        pass

cap.release()

2025-08-06 00:30:47.892 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-06 00:30:48.650 
  command:

    streamlit run /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-08-06 00:30:48.653 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-06 00:30:48.656 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-06 00:30:48.658 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-06 00:30:48.660 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-06 00:30:48.662 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-06 00:30:48.664 Thread 'MainThread': mi